In [1]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

/Users/idilkpkrn/Desktop/llm_data_vis/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Bunny

In [2]:
import open3d as o3d
import numpy as np

bunny = o3d.data.BunnyMesh()
mesh = o3d.io.read_triangle_mesh(bunny.path)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

#pcd = mesh.sample_points_uniformly(number_of_points=500)
#o3d.visualization.draw_geometries([pcd])

2025-03-09 18:16:58.840 Python[14834:23703227] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-09 18:16:58.840 Python[14834:23703227] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False)
vis.add_geometry(mesh)

angles = [0, 45, 90, 135, 180]
for i, angle in enumerate(angles):
    ctr = vis.get_view_control()
    ctr.rotate(angle, 0)
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image(f"view_{i}.png")

vis.destroy_window()

: 

## Segment the Object

In [4]:
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [5]:
from PIL import Image
import torch

image = Image.open("bunny_views/view_0.png")

user_input = "ears of the bunny"

inputs = processor(text=user_input, images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image 
probs = logits_per_image.softmax(dim=1)    


In [8]:
from transformers import Mask2FormerForUniversalSegmentation, Mask2FormerImageProcessor

segmentation_processor = Mask2FormerImageProcessor.from_pretrained("facebook/mask2former-swin-large-coco-instance")
segmentation_model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-coco-instance")

inputs = segmentation_processor(image, return_tensors="pt")
outputs = segmentation_model(**inputs)
outputs
#masks = outputs.masks

/Users/idilkpkrn/Desktop/llm_data_vis/.venv/lib/python3.11/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `Mask2FormerImageProcessor.__init__` and were ignored: '_max_size'
  return func(*args, **kwargs)


Mask2FormerForUniversalSegmentationOutput(loss=None, class_queries_logits=tensor([[[ -4.6909,  -8.7132,  -5.7181,  ...,  -8.2359,  -5.8457,   8.3645],
         [ -3.1940, -10.0483,  -6.6985,  ...,  -8.0570,  -6.6982,   9.2404],
         [ -4.4170,  -9.8760,  -6.8623,  ...,  -8.7166,  -6.1629,   8.8459],
         ...,
         [ -4.4256,  -9.5652,  -6.4327,  ...,  -8.5004,  -6.2744,   8.6360],
         [ -4.6566,  -9.6568,  -6.6830,  ...,  -8.7339,  -6.0780,   8.6415],
         [ -4.7278,  -8.6246,  -6.6024,  ...,  -8.4249,  -5.6185,   8.2067]]],
       grad_fn=<ViewBackward0>), masks_queries_logits=tensor([[[[-13.1014, -14.6546, -17.2258,  ..., -15.8130, -12.0958,  -8.7961],
          [-11.6799, -12.5623, -12.9669,  ...,  -7.5339,  -6.3270,  -8.2004],
          [-12.2438, -12.7873, -12.7143,  ...,  -6.6340,  -6.4419,  -7.5084],
          ...,
          [-15.1115, -11.1915, -10.4298,  ...,  -7.5443,  -8.5253, -10.4924],
          [-15.6806, -12.2614, -11.8428,  ...,  -8.8962,  -9.5624, 

In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, Mask2FormerImageProcessor, Mask2FormerForUniversalSegmentation
import open3d as o3d
import numpy as np

bunny = o3d.data.BunnyMesh()
mesh = o3d.io.read_triangle_mesh(bunny.path)
mesh.compute_vertex_normals()

import torch
from PIL import Image
import numpy as np
import open3d as o3d
from transformers import CLIPProcessor, CLIPModel, Mask2FormerImageProcessor, Mask2FormerForUniversalSegmentation

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

segmentation_processor = Mask2FormerImageProcessor.from_pretrained("facebook/mask2former-swin-large-coco-instance")
segmentation_model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-coco-instance")

def render_views(mesh, num_views=5, image_size=(256, 256)):
    images = []
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=image_size[0], height=image_size[1], visible=False)

    for i in range(num_views):
        vis.add_geometry(mesh)
        ctr = vis.get_view_control()
        ctr.set_zoom(0.8)
        ctr.rotate(60 * i, 0)

        vis.poll_events()
        vis.update_renderer()
        image = vis.capture_screen_float_buffer(do_render=True)
        image = np.asarray(image) * 255
        image = image.astype(np.uint8)
        images.append(Image.fromarray(image))

        vis.clear_geometries()

    vis.destroy_window()
    return images

rendered_images = render_views(mesh)

def segment_image(image, text_prompt):
    inputs = clip_processor(text=text_prompt, images=image, return_tensors="pt", padding=True)
    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image
    #probs = logits_per_image.softmax(dim=1)

    segmentation_inputs = segmentation_processor(image, return_tensors="pt")
    segmentation_outputs = segmentation_model(**segmentation_inputs)

    masks = torch.argmax(segmentation_outputs.class_queries_logits, dim=-1)
    masks = masks.squeeze().cpu().numpy()
    return masks

user_input = "ears of the bunny"

segmented_masks = []
for image in rendered_images:
    masks = segment_image(image, user_input)
    segmented_masks.append(masks)



[array([80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 18, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80]),
 array([80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
        80, 80, 18, 80, 80

In [ ]:
'''
def project_2d_to_3d(mesh, masks, camera_params):
    """
    Project 2D segmentation masks to 3D vertices.
    """
    # Placeholder for projection logic
    # For each vertex in the mesh, check if it projects into the segmented region in any view
    segmented_vertices = set()
    for i, mask in enumerate(masks):
        # Use camera parameters to map 2D mask to 3D vertices
        # (This step requires camera calibration and projection logic)
        pass
    return list(segmented_vertices)

# Placeholder for camera parameters (replace with actual values)
camera_params = [{"position": [0, 0, 5], "orientation": [0, 0, 0]}] * len(rendered_images)

# Project masks to 3D
segmented_vertices = project_2d_to_3d(mesh, segmented_masks, camera_params)

# Step 7: Visualize the segmented 3D mesh
segmented_mesh = mesh.select_by_index(segmented_vertices)
#o3d.visualization.draw_geometries([segmented_mesh])
'''

## Shapes

In [ ]:
import open3d as o3d
import numpy as np

print("Let's define some primitives")
mesh_box = o3d.geometry.TriangleMesh.create_box(width=1.0,
                                                height=1.0,
                                                depth=1.0)
mesh_box.compute_vertex_normals()
mesh_box.paint_uniform_color([0.9, 0.1, 0.1])
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh_sphere.compute_vertex_normals()
mesh_sphere.paint_uniform_color([0.1, 0.1, 0.7])
mesh_cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=0.3,
                                                          height=4.0)
mesh_cylinder.compute_vertex_normals()
mesh_cylinder.paint_uniform_color([0.1, 0.9, 0.1])
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.6, origin=[-2, -2, -2])

[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


Visualizer with name Open3D

In [ ]:
import open3d as o3d
import numpy as np    
knot_data = o3d.data.KnotMesh()
print(f"Reading mesh from file: knot.ply stored at {knot_data.path}")
mesh = o3d.io.read_triangle_mesh(knot_data.path)
mesh = o3d.io.read_triangle_mesh("data/roadBike/roadBike.obj")
print(mesh)

Reading mesh from file: knot.ply stored at /Users/idilkpkrn/open3d_data/extract/KnotMesh/KnotMesh.ply
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
TriangleMesh with 0 points and 0 triangles.


In [ ]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.6, origin=[-2, -2, -2])

In [ ]:
print("We draw a few primitives using collection.")
o3d.visualization.draw_geometries(
    [mesh])

We draw a few primitives using collection.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


2025-02-10 18:08:43.528 Python[49642:2197320] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-10 18:08:43.528 Python[49642:2197320] +[IMKInputSession subclass]: chose IMKInputSession_Modern
